In [ ]:
# !pip install gtts pydub audioop-lts
from gtts import gTTS
from pydub import AudioSegment
import time
import os
import pandas as pd
from datetime import date
import glob
from collections import defaultdict
pd.options.display.max_columns = 100
pd.options.display.max_rows = 130

# 0. Choose setting

In [ ]:
# recording_id, recording_name = '001', '0817_word_p2_k2_sent'
# recording_id, recording_name = '002', '0817_word_p2_k2_csent'
# recording_id, recording_name = '002', '0818_word_p1_k2_csent'
# recording_id, recording_name = '004', '0817_word_p2_k2_wordonly'
# recording_id, recording_name = '005', '0815_word_p2_k2_word_pinyin'
# recording_id, recording_name = '006', '0815_word_p2_k2_ecombo'
# recording_id, recording_name = '007', '0818_recent_p4_k2_sent'
# recording_id, recording_name = '008', '0818_recent_p4_k2_wordonly'
# recording_id, recording_name = '009', '0818_word_p3_k2_sent'
# recording_id, recording_name = '010', '0818_phrase_p2_k2'
# recording_id, recording_name = '012', '0825_eword_p1_k2_csent'

# min_priority, max_priority = 1, 1
# min_known_english_prompt = 2
# max_known_english_prompt = 5
# types_allowed = ['combo', 'no combo', 'two word', 'prefix', 'single char', 'suffix', 'abbreviation']
# types_allowed = ['phrase', 'sentence', 'part sent', 'phrase_save']

In [292]:
recording_id_codes = {
    '001': 'full_word_sent',
    '004': 'word_only',
    '012': 'efirst_csent',
    '006': 'ecombo',
    '013': 'ccombo_sent',
}
WORD_TYPES = ['combo', 'no combo', 'two word', 'prefix', 'single char', 'suffix', 'abbreviation']
PHRASE_TYPES = ['phrase', 'sentence', 'part sent', 'phrase_save']
types_allowed_map = {'_phrase': PHRASE_TYPES, '': WORD_TYPES}
categories_allowed_map = {'animal': ['animal'], '': None}
date_string = date.today().strftime("%m%d")
default_settings = {
    'sort_keys': ['category1', 'pinyin'],
    'sort_asc': [True, True],
}

df_all_recordings_tomake = pd.DataFrame(
    # {
    #     'recording_id': ['001', '004', '001', '004'],
    #     'min_priority': [1, 1, 2, 2],
    #     'max_priority': [1, 1, 2, 2],
    #     'min_known_english_prompt': [3, 3, 3, 3],
    #     'max_known_english_prompt': [5, 5, 5, 5],
    #     'types_allowed_str': ['', '', '', ''],
    #     'min_combo_quality': [5, 5, 5, 5],
    #     'category_type': ['', '', '', ''],
    #  }
)
extra_recordings = [
    # {
    # 'recording_id': '012',
    # 'min_priority': 1,
    # 'max_priority': 1,
    # 'min_known_english_prompt': 3,
    # 'max_known_english_prompt': 5,
    # 'types_allowed_str': '',
    # 'min_combo_quality': 5,
    # 'category_type': '',
    # },
    # {
    # 'recording_id': '012',
    # 'min_priority': 2,
    # 'max_priority': 2,
    # 'min_known_english_prompt': 3,
    # 'max_known_english_prompt': 5,
    # 'types_allowed_str': '',
    # 'min_combo_quality': 5,
    # 'category_type': '',
    # },
    # {
    # 'recording_id': '004',
    # 'min_priority': 1,
    # 'max_priority': 2,
    # 'min_known_english_prompt': 2,
    # 'max_known_english_prompt': 5,
    # 'types_allowed_str': '_phrase',
    # 'min_combo_quality': 5,
    # 'category_type': '',
    # },
    # {
    # 'recording_id': '006',
    # 'min_priority': 1,
    # 'max_priority': 2,
    # 'min_known_english_prompt': 2,
    # 'max_known_english_prompt': 5,
    # 'types_allowed_str': '',
    # 'min_combo_quality': 5,
    # 'category_type': '',
    # },
    # {
    # 'recording_id': '006',
    # 'min_priority': 1,
    # 'max_priority': 2,
    # 'min_known_english_prompt': 2,
    # 'max_known_english_prompt': 5,
    # 'types_allowed_str': '',
    # 'min_combo_quality': 5,
    # 'category_type': '',
    # },
    {
    'recording_id': '013',
    'min_priority': 1,
    'max_priority': 4,
    'min_known_english_prompt': 1,
    'max_known_english_prompt': 5,
    'types_allowed_str': '',
    'min_combo_quality': 2,
    'category_type': 'animal',
    'sort_keys': ['category2', 'quality', 'pinyin'],
    'sort_asc': [True, True, True],
    },
]
df_all_recordings_tomake = pd.concat([df_all_recordings_tomake, pd.DataFrame(extra_recordings)], ignore_index=True)
df_all_recordings_tomake['recording_id_code'] = df_all_recordings_tomake['recording_id'].map(recording_id_codes)
df_all_recordings_tomake['types_allowed'] = df_all_recordings_tomake['types_allowed_str'].map(types_allowed_map)
df_all_recordings_tomake['categories_allowed'] = df_all_recordings_tomake['category_type'].map(categories_allowed_map)
df_all_recordings_tomake['recording_name'] = df_all_recordings_tomake.apply(lambda x: f"{date_string}_{x['recording_id']}{x['types_allowed_str']}_{x['recording_id_code']}_p{x['min_priority']}_{x['max_priority']}_k{x['min_known_english_prompt']}", axis=1)

for setting_key, setting_default in default_settings.items():
    if setting_key not in df_all_recordings_tomake.columns:
        df_all_recordings_tomake[setting_key] = [setting_default]*len(df_all_recordings_tomake)
    else:
        df_all_recordings_tomake[setting_key] = df_all_recordings_tomake[setting_key].apply(lambda d: d if isinstance(d, list) else setting_default)
df_all_recordings_tomake

,recording_id,min_priority,max_priority,min_known_english_prompt,max_known_english_prompt,types_allowed_str,min_combo_quality,category_type,sort_keys,sort_asc,recording_id_code,types_allowed,categories_allowed,recording_name
0,013,1,4,1,5,,2,animal,"[category2, quality, pinyin]","[True, True, True]",ccombo_sent,"[combo, no combo, two word, prefix, single cha...",[animal],0826_013_ccombo_sent_p1_4_k1


# 1. Load data

In [293]:
cols_keep = [
    'id', 'chinese', 'pinyin', 'english',
    'type', 'priority', 'known', 'known_pinyin_prompt', 'known_english_prompt',
    'phonetic', 'category1', 'category2', 'quality',
    'word1', 'word1_english', 'word2', 'word2_english', 'word3', 'word3_english', 'word4', 'word4_english',
    'sentence', 'sentence_pinyin', 'sentence_english', 'date']
sheet_url = 'https://docs.google.com/spreadsheets/d/1pw9EAIvtiWenPDBFBIf7pwTh0FvIbIR0c3mY5gJwlDk/edit#gid=0'
sheet_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(sheet_url)[cols_keep]
df = df.dropna(subset=['chinese', 'english'])
df['known_english_prompt'] = df['known_english_prompt'].fillna(6)
df['quality'] = df['quality'].fillna(5)
print(df.shape)
df.head()

(4973, 25)


,id,chinese,pinyin,english,type,priority,known,known_pinyin_prompt,known_english_prompt,phonetic,category1,category2,quality,word1,word1_english,word2,word2_english,word3,word3_english,word4,word4_english,sentence,sentence_pinyin,sentence_english,date
0,2772,后缀,hòu zhuì,suffix,combo,2.0,5.0,2.0,5.0,NaN,language,NaN,2.0,后,after,点缀,embellishment,NaN,NaN,NaN,NaN,“-ness”是一个常见的后缀,“-ness” shì yī gè chángjiàn de hòuzhuì,-ness is a common suffix,NaN
1,619,习语,xí yǔ,common saying;idiom,combo,2.0,5.0,5.0,5.0,NaN,language,NaN,2.0,习惯,habit,语,language,NaN,NaN,NaN,NaN,“一箭双雕”是一个习语,“Yī jiàn shuāng diāo” shì yī gè xíyǔ,“Kill two birds with one stone” is an idiom,NaN
2,1310,音译词,yīn yì cí,transliteration,combo,3.0,5.0,3.0,5.0,NaN,language,NaN,2.0,音,sound,翻译,to translate,词,word,NaN,NaN,“咖啡”是一个音译词,“kāfēi” shì yī gè yīnyìcí,Coffee is a transliteration,NaN
3,2818,正常,zhèng ​cháng,normal,no combo,1.0,3.0,2.0,3.0,NaN,adjective,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,一切都很正常,Yīqiè dōu hěn zhèngcháng,Everything is normal,NaN
4,455,口水,kǒu shuǐ,saliva,combo,2.0,3.0,1.0,2.0,NaN,food,NaN,1.0,口,mouth,水,water,NaN,NaN,NaN,NaN,一想到美食他就流口水,Yī xiǎngdào měishí tā jiù liú kǒushuǐ,He drools when he thinks of delicious food,NaN


# 2. Select data

In [294]:
def filter_by_recording_type(df, recording_id):
    """Filter the DataFrame based on the recording type."""
    if recording_id in ['001', '009']:
        return df.dropna(subset=['sentence', 'sentence_english'])
    elif recording_id in ['002', '012', '013']:
        return df.dropna(subset=['sentence'])
    elif recording_id in ['004', '010']:
        return df.dropna(subset=['chinese', 'english'])
    elif recording_id == '005':
        return df.dropna(subset=['pinyin'])
    elif recording_id == '006':
        return df.dropna(subset=['word1', 'word1_english', 'word2', 'word2_english'])
    elif recording_id == '007':
        return df[df['date'] >= '2025-07-15'].dropna(subset=['sentence', 'sentence_english'])
    elif recording_id == '008':
        return df[df['date'] >= '2025-07-15']
    else:
        raise ValueError("Invalid recording ID")

# Which data to use this time?
dfs_by_recording = {}
for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    dfs_by_recording[rrow['recording_name']] = df[
            (df['priority'] <= rrow['max_priority']) &
            (df['priority'] >= rrow['min_priority']) &
            (df['known_english_prompt'] >= rrow['min_known_english_prompt']) &
            (df['known_english_prompt'] <= rrow['max_known_english_prompt']) &
            (df['quality'] <= rrow['min_combo_quality']) &
            (df['type'].isin(rrow['types_allowed'])) &
            (df['category1'].isin(rrow['categories_allowed']) if rrow['categories_allowed'] is not None else True)
        ]
    dfs_by_recording[rrow['recording_name']] = (filter_by_recording_type(dfs_by_recording[rrow['recording_name']], rrow['recording_id'])
        .sort_values(rrow['sort_keys'], ascending=rrow['sort_asc'],)
        .reset_index(drop=True))
    print(rrow['recording_name'], dfs_by_recording[rrow['recording_name']].shape)

dfs_by_recording[rrow['recording_name']].head(10)

0826_013_ccombo_sent_p1_4_k1 (18, 25)


,id,chinese,pinyin,english,type,priority,known,known_pinyin_prompt,known_english_prompt,phonetic,category1,category2,quality,word1,word1_english,word2,word2_english,word3,word3_english,word4,word4_english,sentence,sentence_pinyin,sentence_english,date
0,4084,公鸡,gōng jī,rooster,combo,3.0,5.0,5.0,5.0,NaN,animal,bird,2.0,老公,husband,鸡,chicken,NaN,NaN,NaN,NaN,公鸡每天早上打鸣,gong ji mei tian zao shang da ming,the rooster crows every morning,2025-07-15
1,4083,母鸡,mǔ jī,hen,combo,3.0,5.0,5.0,5.0,NaN,animal,bird,2.0,母,mother,鸡,chicken,NaN,NaN,NaN,NaN,母鸡正在下蛋,mu ji zheng zai xia dan,the hen is laying eggs,2025-07-15
2,169,飞虫,fēi​ chóng,fly;flying insect,combo,1.0,2.0,1.0,2.0,NaN,animal,bug,2.0,飞,to fly,虫子,bug,NaN,NaN,NaN,NaN,房间里飞进了一只飞虫,Fángjiān lǐ fēi jìn le yī zhī fēichóng,A bug flew into the room,NaN
3,4517,黄蜂,huáng fēng,wasp,combo,4.0,5.0,5.0,5.0,NaN,animal,bug,2.0,黄,yellow,蜜蜂,bee,NaN,NaN,NaN,NaN,花园里有一只黄蜂,Huāyuán lǐ yǒu yì zhī huángfēng,There is a wasp in the garden,2025-08-17
4,1052,黄牛,huáng niú,cattle;scalper,combo,2.0,1.0,1.0,2.0,NaN,animal,domestic,2.0,黄,yellow,牛,cow,NaN,NaN,NaN,NaN,票被黄牛炒得很贵,Piào bèi huángniú chǎo de hěn guì,Scalpers made the tickets very expensive,NaN
5,544,金毛,jīn máo,golden retriever,combo,2.0,3.0,1.0,1.0,NaN,animal,domestic,2.0,金,gold,毛,hair,NaN,NaN,NaN,NaN,他家养了一只金毛,Tā jiā yǎng le yī zhī jīnmáo,His family has a golden retriever,NaN
6,3763,毛皮,máo pí,fur,combo,2.0,5.0,5.0,2.0,NaN,animal,nonanimal,1.0,毛,hair,皮,skin,NaN,NaN,NaN,NaN,毛皮外套很保暖,Máopí wàitào hěn bǎonuǎn,Fur coats are very warm,NaN
7,4487,汪汪,wāng wāng,woof woof,no combo,2.0,5.0,5.0,5.0,NaN,animal,nonanimal,1.0,字,character,身体,body,NaN,NaN,NaN,NaN,小狗在门口汪汪叫,Xiǎo gǒu zài ménkǒu wāngwāng jiào,The little dog is barking at the door,2025-08-14
8,1201,项圈,xiàng quān,collar (for a dog),combo,2.0,5.0,2.0,5.0,NaN,animal,nonanimal,1.0,项,neck,圈,circle,NaN,NaN,NaN,NaN,狗戴着项圈,Gǒu dàizhe xiàngquān,The dog is wearing a collar,NaN
9,4330,抚摸,fǔ mō,to pet;caress,combo,2.0,5.0,5.0,5.0,NaN,animal,nonanimal,2.0,抚慰,to soothe,摸,to touch (with hand),NaN,NaN,NaN,NaN,他轻轻抚摸小猫,Tā qīngqīng fǔmō xiǎo māo,He gently strokes the kitten,2025-07-28


In [243]:
def pinyin_to_tones(pinyin):
    """Convert pinyin to tones."""
    pinyin = pinyin.replace('ā', 'a1').replace('á', 'a2').replace('ǎ', 'a3').replace('à', 'a4')
    pinyin = pinyin.replace('ē', 'e1').replace('é', 'e2').replace('ě', 'e3').replace('è', 'e4')
    pinyin = pinyin.replace('ī', 'i1').replace('í', 'i2').replace('ǐ', 'i3').replace('ì', 'i4')
    pinyin = pinyin.replace('ō', 'o1').replace('ó', 'o2').replace('ǒ', 'o3').replace('ò', 'o4')
    pinyin = pinyin.replace('ū', 'u1').replace('ú', 'u2').replace('ǔ', 'u3').replace('ù', 'u4')
    pinyin = pinyin.replace('ü', 'v1').replace('ǘ', 'v2').replace('ǚ', 'v3').replace('ǜ', 'v4')
    tones = []
    for pinyin_oneword in pinyin.split():
        tone = '1'  # Default tone
        for i in range(1, 5):
            if f'{i}' in pinyin_oneword:
                tone = str(i)
                break
        tones.append(tone)
    return tones

for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    if rrow['recording_id'] == '005':
        dfs_by_recording[rrow['recording_name']]['pinyin_tones'] = dfs_by_recording[rrow['recording_name']]['pinyin'].apply(pinyin_to_tones)
        dfs_by_recording[rrow['recording_name']][['pinyin', 'pinyin_tones']]

        # Make pinyin audio
        pinyin_tones = ['1', '2', '3', '4']
        for tone_str in pinyin_tones:
            gTTS(tone_str, lang='en').save(f"audio_files/english/{tone_str}.mp3")

# 3. TTS

In [298]:
def create_tts_file(tts_type, content_str, lang_name, last_timestamp, chinese_char, recording_id):
    if tts_type == 'zh_slow':
        slow_mode = True
    else:
        slow_mode = False

    new_file_path = f"audio_files/{tts_type}/{content_str}.mp3"
    if not os.path.exists(new_file_path):
        # Delete final row file, if exists, since will have to rewrite it
        row_file_path = f"audio_files/rows/{recording_id}/{chinese_char}.mp3"
        if os.path.exists(row_file_path):
            os.remove(row_file_path)
        
        try:
            gTTS(content_str, lang=lang_name, slow=slow_mode).save(new_file_path)
        except:
            # Wait 60 seconds and try again
            print(f"!!!!!!! FAILURE, wait 52 seconds, row{i_row}, {tts_type}, {content_str} !!!!!!!")
            time.sleep(52)
            try:
                gTTS(content_str, lang=lang_name, slow=slow_mode).save(new_file_path)
            except:
                # Wait 60 seconds and try again
                print(f"!!!!!!!!!! FAILURE AGAIN, wait 278 seconds, row{i_row}, {tts_type}, {content_str} !!!!!!!!!!")
                time.sleep(278)
                gTTS(content_str, lang=lang_name, slow=slow_mode).save(new_file_path)
        print(f"{(time.time()-last_timestamp):.3f}s, recid{recording_id}, row{i_row}, {tts_type}, {content_str}")
    else:
        print(f"{(time.time()-last_timestamp):.3f}s, ALREADY EXISTS, recid{recording_id}, row{i_row}, {tts_type}, {content_str}")


for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    for i_row, row in dfs_by_recording[rrow['recording_name']].iterrows():
        create_tts_file(tts_type='zh', content_str=row['chinese'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
        create_tts_file(tts_type='zh_slow', content_str=row['chinese'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
        create_tts_file(tts_type='english', content_str=row['english'], lang_name='en', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
        # create_tts_file(tts_type='zh_tw', content_str=row['chinese'], lang_name='zh-tw', last_timestamp=time.time())
        if rrow['recording_id'] in ['001', '007', '009', '013']:
            create_tts_file(tts_type='zh', content_str=row['sentence'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
            create_tts_file(tts_type='english', content_str=row['sentence_english'], lang_name='en', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
        if rrow['recording_id'] in ['002', '011', '012', '013']:
            create_tts_file(tts_type='zh', content_str=row['sentence'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
        if rrow['recording_id'] in ['006', '013']:
            create_tts_file(tts_type='zh', content_str=row['word1'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
            create_tts_file(tts_type='zh', content_str=row['word2'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
            create_tts_file(tts_type='english', content_str=row['word1_english'], lang_name='en', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
            create_tts_file(tts_type='english', content_str=row['word2_english'], lang_name='en', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
            if not pd.isna(row['word3']):
                create_tts_file(tts_type='zh', content_str=row['word3'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
                create_tts_file(tts_type='english', content_str=row['word3_english'], lang_name='en', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
            if not pd.isna(row['word4']):
                create_tts_file(tts_type='zh', content_str=row['word4'], lang_name='zh-cn', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])
                create_tts_file(tts_type='english', content_str=row['word4_english'], lang_name='en', last_timestamp=time.time(), chinese_char=row['chinese'], recording_id=rrow['recording_id'])

0.000s, ALREADY EXISTS, recid013, row0, zh, 公鸡
0.000s, ALREADY EXISTS, recid013, row0, zh_slow, 公鸡
0.000s, ALREADY EXISTS, recid013, row0, english, rooster
0.000s, ALREADY EXISTS, recid013, row0, zh, 公鸡每天早上打鸣
0.000s, ALREADY EXISTS, recid013, row0, english, the rooster crows every morning
0.000s, ALREADY EXISTS, recid013, row0, zh, 公鸡每天早上打鸣
1.240s, recid013, row0, zh, 老公
1.167s, recid013, row0, zh, 鸡
0.000s, ALREADY EXISTS, recid013, row0, english, husband
1.193s, recid013, row0, english, chicken
0.000s, ALREADY EXISTS, recid013, row1, zh, 母鸡
0.000s, ALREADY EXISTS, recid013, row1, zh_slow, 母鸡
0.000s, ALREADY EXISTS, recid013, row1, english, hen
0.000s, ALREADY EXISTS, recid013, row1, zh, 母鸡正在下蛋
0.000s, ALREADY EXISTS, recid013, row1, english, the hen is laying eggs
0.000s, ALREADY EXISTS, recid013, row1, zh, 母鸡正在下蛋
0.000s, ALREADY EXISTS, recid013, row1, zh, 母
0.000s, ALREADY EXISTS, recid013, row1, zh, 鸡
0.000s, ALREADY EXISTS, recid013, row1, english, mother
0.000s, ALREADY EXISTS, 

# 4. Combine files for individual vocab words

In [ ]:
def load_one_audio_from_path(mp3_path):
    try:
        audio = AudioSegment.from_mp3(mp3_path)
    except FileNotFoundError:
        # NOTE: THIS IS NOT THE RIGHT EXCEPTION. I FORGET.
        os.remove(mp3_path)
        audio = AudioSegment.from_mp3(mp3_path)
    return audio


def load_audio(recording_id, row):
    dict_audio_durations = defaultdict(list)
    chinese_audio = load_one_audio_from_path(f"audio_files/zh/{row['chinese']}.mp3")
    chinese_slow_audio = load_one_audio_from_path(f"audio_files/zh_slow/{row['chinese']}.mp3")
    english_audio = load_one_audio_from_path(f"audio_files/english/{row['english']}.mp3")
    
    if recording_id in ['001', '007', '009']:
        sent_audio = load_one_audio_from_path(f"audio_files/zh/{row['sentence']}.mp3")
        sent_english_audio = load_one_audio_from_path(f"audio_files/english/{row['sentence_english']}.mp3")
        combined = chinese_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_500ms + sent_audio + pause_500ms + sent_english_audio + pause_500ms + sent_audio + pause_1000ms

    elif recording_id in ['002', '011']:
        sent_audio = load_one_audio_from_path(f"audio_files/zh/{row['sentence']}.mp3")
        combined = chinese_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_500ms + sent_audio + pause_500ms + sent_audio + pause_1000ms

    elif recording_id in ['004', '008', '010']:
        combined = chinese_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_1000ms

    elif recording_id == '005':
        tones_audio = AudioSegment.silent(duration=0)
        for pinyin_tone in row['pinyin_tones']:
            tones_audio += load_one_audio_from_path(f"audio_files/english/{pinyin_tone}.mp3")
            tones_audio += pause_100ms

        combined = chinese_audio + pause_500ms + tones_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_1000ms

    elif recording_id == '006':
        word1_audio = load_one_audio_from_path(f"audio_files/zh/{row['word1']}.mp3")
        word1e_audio = load_one_audio_from_path(f"audio_files/english/{row['word1_english']}.mp3")
        word2_audio = load_one_audio_from_path(f"audio_files/zh/{row['word2']}.mp3")
        word2e_audio = load_one_audio_from_path(f"audio_files/english/{row['word2_english']}.mp3")
        if not pd.isna(row['word3']):
            word3_audio = load_one_audio_from_path(f"audio_files/zh/{row['word3']}.mp3")
            word3e_audio = load_one_audio_from_path(f"audio_files/english/{row['word3_english']}.mp3")
        if not pd.isna(row['word4']):
            word4_audio = load_one_audio_from_path(f"audio_files/zh/{row['word4']}.mp3")
            word4e_audio = load_one_audio_from_path(f"audio_files/english/{row['word4_english']}.mp3")

        component_words_audio = word1_audio + pause_100ms + word1e_audio + pause_500ms + word2_audio + pause_100ms + word2e_audio
        if not pd.isna(row['word3']):
            component_words_audio += pause_500ms + word3_audio + pause_100ms + word3e_audio
        if not pd.isna(row['word4']):
            component_words_audio += pause_500ms + word4_audio + pause_100ms + word4e_audio
        combined = chinese_audio + pause_500ms + component_words_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_500ms

    elif recording_id == '013':
        sent_audio = load_one_audio_from_path(f"audio_files/zh/{row['sentence']}.mp3")
        sent_english_audio = load_one_audio_from_path(f"audio_files/english/{row['sentence_english']}.mp3")
        word1_audio = load_one_audio_from_path(f"audio_files/zh/{row['word1']}.mp3")
        word1e_audio = load_one_audio_from_path(f"audio_files/english/{row['word1_english']}.mp3")
        word2_audio = load_one_audio_from_path(f"audio_files/zh/{row['word2']}.mp3")
        word2e_audio = load_one_audio_from_path(f"audio_files/english/{row['word2_english']}.mp3")
        if not pd.isna(row['word3']):
            word3_audio = load_one_audio_from_path(f"audio_files/zh/{row['word3']}.mp3")
            word3e_audio = load_one_audio_from_path(f"audio_files/english/{row['word3_english']}.mp3")
        if not pd.isna(row['word4']):
            word4_audio = load_one_audio_from_path(f"audio_files/zh/{row['word4']}.mp3")
            word4e_audio = load_one_audio_from_path(f"audio_files/english/{row['word4_english']}.mp3")

        component_words_audio = word1_audio + pause_100ms + word1e_audio + pause_500ms + word2_audio + pause_100ms + word2e_audio
        if not pd.isna(row['word3']):
            component_words_audio += pause_500ms + word3_audio + pause_100ms + word3e_audio
        if not pd.isna(row['word4']):
            component_words_audio += pause_500ms + word4_audio + pause_100ms + word4e_audio
        combined = chinese_audio + pause_500ms + component_words_audio + pause_500ms + chinese_slow_audio + pause_500ms + english_audio + pause_500ms + sent_audio + pause_500ms + sent_english_audio + pause_1000ms

        dict_audio_durations['chinese'].append(chinese_audio.duration_seconds)
        dict_audio_durations['component_words'].append(component_words_audio.duration_seconds)
        dict_audio_durations['chinese_slow'].append(chinese_slow_audio.duration_seconds)
        dict_audio_durations['english'].append(english_audio.duration_seconds)
        dict_audio_durations['sent'].append(sent_audio.duration_seconds)
        dict_audio_durations['sent_english'].append(sent_english_audio.duration_seconds)
        # dict_audio_durations['concat_actual'].append(sent_english_audio.duration_seconds)
        # dict_audio_durations['sum_theory'].append(sent_english_audio.duration_seconds)

        # durations_dict['duration_sent_c'].append(AudioSegment.from_mp3(f"audio_files/zh/{row['sentence']}.mp3").duration_seconds)
        # durations_dict['duration_sent_e'].append(AudioSegment.from_mp3(f"audio_files/english/{row['sentence_english']}.mp3").duration_seconds)
        # durations_dict['duration_concat'].append(AudioSegment.from_mp3(f"audio_files/rows/{rrow['recording_id']}/{row['chinese']}.mp3").duration_seconds)
        # durations_dict['duration_sum'].append(durations_dict['duration_chinese'][-1] + durations_dict['duration_chinese_slow'][-1] + durations_dict['duration_english'][-1] + durations_dict['duration_sent_e'][-1] + 2*durations_dict['duration_sent_c'][-1] + 2.5 + 1)

    elif recording_id == '012':
        sent_audio = AudioSegment.from_mp3(f"audio_files/zh/{row['sentence']}.mp3")
        combined = english_audio + pause_500ms + chinese_audio + pause_500ms + sent_audio + pause_1000ms

    else:
        raise ValueError(f"Invalid recording_id: {recording_id}")
    
    df_audio_durations = pd.DataFrame(dict_audio_durations)
    return combined, df_audio_durations

pause_100ms = AudioSegment.silent(duration=100)
pause_500ms = AudioSegment.silent(duration=500)
pause_1000ms = AudioSegment.silent(duration=1000)

dfs_audio_durations = []
for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    for i_row, row in dfs_by_recording[rrow['recording_name']].iterrows():
        start_time = time.time()
        new_folder_path = f"audio_files/rows/{rrow['recording_id']}"
        new_file_path = f"{new_folder_path}/{row['chinese']}.mp3"
        os.makedirs(new_folder_path, exist_ok=True)
        if not os.path.exists(new_file_path):
            combined, df_audio_durations_onerow = load_audio(rrow['recording_id'], row)
            dfs_audio_durations.append(df_audio_durations_onerow)
            combined.export(new_file_path, format="mp3")
            print(f"{(time.time()-start_time):.2f} seconds, recid{rrow['recording_name']}, row {i_row}, {row['chinese']}")

        else:
            print(f"{(time.time()-start_time):.2f} seconds, recid{rrow['recording_name']}, row {i_row}, {row['chinese']} ALREADY EXISTS")

df_audio_durations = pd.concat(dfs_audio_durations, ignore_index=True)


0.84 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 0, 公鸡
0.78 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 1, 母鸡
0.77 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 2, 飞虫
0.88 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 3, 黄蜂
0.76 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 4, 黄牛
0.78 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 5, 金毛
0.77 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 6, 毛皮
0.76 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 7, 汪汪
0.80 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 8, 项圈
0.83 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 9, 抚摸
0.97 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 10, 八爪鱼
0.79 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 11, 龙虾
0.81 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 12, 郊狼
0.80 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 13, 山羊
1.08 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 14, 长颈鹿
0.77 seconds, recid0826_013_ccombo_sent_p1_4_k1, row 15, 恐龙
0.76 seconds, recid0826_013_ccombo_sent_p1_4_k1,

In [315]:
df_audio_durations

,chinese,chinese_slow,english
0,1.128,1.320,1.008
1,1.128,1.296,0.840
2,1.248,1.416,2.136
3,1.128,1.296,0.936
4,1.200,1.392,1.896
5,1.104,1.272,1.536
6,1.104,1.272,0.816
7,1.128,1.296,1.224
8,1.248,1.464,1.968
9,1.152,1.320,1.872


# 5. Combine individual vocab into final audio recording

In [301]:
# Combine the audio files into a single file
# for i_rrow, rrow in df_all_recordings_tomake.iterrows():
for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    start_time = time.time()
    all_audio_files = []
    for i_row, row in dfs_by_recording[rrow['recording_name']].iterrows():
        audio_file = f"audio_files/rows/{rrow['recording_id']}/{row['chinese']}.mp3"
        if os.path.exists(audio_file):
            all_audio_files.append(AudioSegment.from_mp3(audio_file))
        else:
            print(f"Missing file: {audio_file}")

    combined = all_audio_files[0]
    for audio in all_audio_files[1:]:
        combined += audio
    combined.export(f"audio_files/products/{rrow['recording_name']}.mp3", format="mp3")
    print(f"{(time.time()-start_time):.2f}s, audio_files/products/{rrow['recording_name']}.mp3")

2.17s, audio_files/products/0826_013_ccombo_sent_p1_4_k1.mp3


# 6. video recording

In [305]:
dfs_by_recording.keys()

dict_keys(['0826_013_ccombo_sent_p1_4_k1'])

## 6a. create individual clips

In [316]:

from PIL import Image, ImageDraw, ImageFont
from moviepy import ImageClip, CompositeVideoClip, AudioFileClip

In [ ]:
def create_text_clip(text, duration, timestamp_start, image_file_name, size=(1280, 720), bg_color='white', text_color='black', font_path=hanzi_font_path):
    # Use textcip
    # clip_c = TextClip(text=f"{chinese}\n{pinyin}", font='Marker Felt', font_size=70, color=text_color, bg_color=bg_color, size=size, method='label', duration=duration_text_c)
    img = Image.new("RGB", size, color=bg_color)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, 70)
    draw.multiline_text((100, 200), text, font=font, fill=text_color, spacing=30)
    img.save(f"audio_files/img_for_video/{image_file_name}.png")
    clip = ImageClip(f"audio_files/img_for_video/{image_file_name}.png", duration=duration).with_start(timestamp_start)
    return clip
    
# Create all clips
clips = []
for idx, row in df_video.iterrows():
    clips.append(create_text_clip(f"{row['chinese']}\n{row['pinyin']}", row['duration_text_c'], row['timestamp_c'], f"{row['chinese']}_chinese"))
    clips.append(create_text_clip(f"{row['chinese']}\n{row['pinyin']}\n{row['english']}", row['duration_text_e'], row['timestamp_e'], f"{row['chinese']}_english"))
    clips.append(create_text_clip(f"{row['sentence']}\n{row['sentence_pinyin']}", row['duration_sent_c'], row['timestamp_sent_c'], f"{row['chinese']}_sentence"))
    clips.append(create_text_clip(f"{row['sentence']}\n{row['sentence_pinyin']}\n{row['sentence_english']}", row['duration_sent_e'], row['timestamp_sent_e'], f"{row['chinese']}_sentence_e"))
    print(f'{idx}: {row["chinese"]}')

clips[1].display_in_notebook() 

0: 公鸡
1: 母鸡
2: 飞虫
3: 黄蜂
4: 黄牛
5: 金毛
6: 毛皮
7: 汪汪
8: 项圈
9: 抚摸
10: 八爪鱼
11: 龙虾
12: 郊狼
13: 山羊
14: 长颈鹿
15: 恐龙
16: 熊猫
17: 斑马


,chinese,pinyin,english,sentence,sentence_pinyin,sentence_english,duration_chinese,duration_chinese_slow,duration_english,duration_sent_c,duration_sent_e,duration_concat,duration_sum,timestamp_end,timestamp_c,timestamp_e,timestamp_sent_c,timestamp_sent_e,duration_text_c,duration_text_e
0,公鸡,gōng jī,rooster,公鸡每天早上打鸣,gong ji mei tian zao shang da ming,the rooster crows every morning,1.128,1.320,1.008,3.212,8.342917,16.510917,14.756,16.510917,0.000000,3.448000,4.956000,8.168000,3.448,1.508
1,母鸡,mǔ jī,hen,母鸡正在下蛋,mu ji zheng zai xia dan,the hen is laying eggs,1.128,1.296,0.840,2.636,7.766917,15.166917,13.052,31.677833,16.510917,19.934917,21.274917,23.910917,3.424,1.340
2,飞虫,fēi​ chóng,fly;flying insect,房间里飞进了一只飞虫,Fángjiān lǐ fēi jìn le yī zhī fēichóng,A bug flew into the room,1.248,1.416,2.136,3.764,7.886917,17.950917,16.892,49.628750,31.677833,35.341833,37.977833,41.741833,3.664,2.636
3,黄蜂,huáng fēng,wasp,花园里有一只黄蜂,Huāyuán lǐ yǒu yì zhī huángfēng,There is a wasp in the garden,1.128,1.296,0.936,3.212,8.030917,16.102917,14.396,65.731667,49.628750,53.052750,54.488750,57.700750,3.424,1.436
4,黄牛,huáng niú,cattle;scalper,票被黄牛炒得很贵,Piào bèi huángniú chǎo de hěn guì,Scalpers made the tickets very expensive,1.200,1.392,1.896,3.380,8.894917,18.262917,16.892,83.994583,65.731667,69.323667,71.719667,75.099667,3.592,2.396
5,金毛,jīn máo,golden retriever,他家养了一只金毛,Tā jiā yǎng le yī zhī jīnmáo,His family has a golden retriever,1.104,1.272,1.536,3.044,8.198917,16.654917,15.236,100.649500,83.994583,87.370583,89.406583,92.450583,3.376,2.036
6,毛皮,máo pí,fur,毛皮外套很保暖,Máopí wàitào hěn bǎonuǎn,Fur coats are very warm,1.104,1.272,0.816,2.972,7.790917,15.454917,13.772,116.104417,100.649500,104.025500,105.341500,108.313500,3.376,1.316
7,汪汪,wāng wāng,woof woof,小狗在门口汪汪叫,Xiǎo gǒu zài ménkǒu wāngwāng jiào,The little dog is barking at the door,1.128,1.296,1.224,3.428,8.726917,17.302917,15.596,133.407333,116.104417,119.528417,121.252417,124.680417,3.424,1.724
8,项圈,xiàng quān,collar (for a dog),狗戴着项圈,Gǒu dàizhe xiàngquān,The dog is wearing a collar,1.248,1.464,1.968,2.276,7.934917,16.390917,13.892,149.798250,133.407333,137.119333,139.587333,141.863333,3.712,2.468
9,抚摸,fǔ mō,to pet;caress,他轻轻抚摸小猫,Tā qīngqīng fǔmō xiǎo māo,He gently strokes the kitten,1.152,1.320,1.872,3.116,9.206917,18.166917,15.308,167.965167,149.798250,153.270250,155.642250,158.758250,3.472,2.372


## 6b. Create video with audio

In [ ]:
# Load audio
audio = AudioFileClip(f"audio_files/products/{recording_id}_{recording_name}.mp3")
audio.duration

# Compare durations of audio and video
all_clips_duration = sum(clip.duration for clip in clips)
print(f"Audio: {audio.duration:.3f} seconds")
print(f"Video: {all_clips_duration:.3f} seconds")

In [ ]:
# Combine clips
video = CompositeVideoClip(clips, size=(1280,720))
video.audio = audio
video.duration = audio.duration

# Export video
video.write_videofile(f"audio_files/products/{recording_id}_{recording_name}.mp4", fps=24)

# 6. temp write file from pdf

In [ ]:
file_texts = [
    """
第一部分 马克思主义哲学原理
考点1 哲学的基本问题
恩格斯指出：“全部哲学，特别是近代哲学的重大的基本问题，是思维和存在的关系问
题。”
考点2 哲学的主要派别
    """,
    """
（1）哲学的基本派别特指唯物主义与唯心主义两大派别。对哲学基本问题第一方面的
不同回答是划分哲学基本派别的唯一标准。
    """,
    """
（2）唯物主义有三种历史形态，即古代朴素唯物主义、近代形而上学唯物主义和马克
思主义的辩证唯物主义与历史唯物主义。唯心主义则有两种基本形态，即主观唯心主义和客
观唯心主义。
    """,
    """
考点3 物质及其存在形态
（1）列宁关于物质的定义：物质是标志客观实在的哲学范畴。这就指出了客观实在性
是物质的唯一特性。
（2）运动是物质的根本属性和存在方式。
（3）时空是运动着的物质的存在形式。
    """,
    """
考点4 物质与意识的辩证关系
（1）物质决定意识，意识对物质具有反作用，这种反作用就是意识的能动作用。
（2）客观规律和发挥人的主观能动性既是对立的，又是统一的，它们相互依存并相互
转化。
    """,
    """
考点5 主观能动性与客观规律性的统一
客观规律和发挥人的主观能动性既是对立的，又是统一的，它们相互依存并相互转化。
客观规律和主观能动性相互依存有两层含义：一方面，主观能动性依赖于客观规律性。没有
客观规律性，就没有主观能动性可言。客观规律性是主观能动性的前提和基础。另一方面，
规律是客观的，是不以人的意志为转移的，人们既不能创造规律也不能消灭规律，但是，它
并不排斥和否定人的主观能动性，因为客观规律可以被人们的意识所认识、所利用。
正确发挥人的主观能动性的前提和条件：
    """,
    """
（1）正确发挥人的主观能动性的前提是从一切实际出发。
（2）正确发挥人的主观能动性的基本途径是实践。
（3）正确发挥人的主观能动性，还要依赖于一定的物质条件和物质手段。否则，人的
意识再“巧”也创造不出任何物质的东西来。
    """,
    """
考点6 唯物辩证法的基本规律
（1）对立统一规律是唯物辩证法的实质和核心。对立统一规律揭示了事物普遍联系的
根本内容和变化发展的内在动力，从根本上回答了事物为什么会发展的问题；对立统一规律
是贯穿量变质变规律、否定之否定规律以及唯物辩证法基本范畴的中心线索，也是理解这些
规律和范畴的“钥匙”；对立统一规律提供了人们认识世界和改造世界的根本方法——矛盾
分析方法。
    """,
    """
（2）量变和质变的辩证关系是：第一，量变是质变的必要准备。任何事物的变化都有
一个量变的积累过程，没有量变的积累，质变就不会发生。第二，质变是量变的必然结果。
单纯的量变不会永远持续下去，量变达到一定程度必然引起质变。第三，量变和质变是相互
渗透的。一方面，在总的量变过程中有阶段性和局部性的部分质变；另一方面，在质变过程
中也有旧质在量上的收缩和新质在量上的扩张。量变和质变是相互依存、相互贯通的，量变
引起质变，在新质的基础上，事物又开始新的量变，如此交替循环，构成了事物的发展过程。
量变质变规律体现了事物发展的渐进性和飞跃性的统一。
    """,
    """
（3）否定之否定规律揭示了事物发展的方向和道路。由于内部矛盾的推动，事物由自
我肯定状态到达自我否定状态，进而再由这种否定状态发展到新的否定即否定之否定状态，
从而开辟出自我发展和自我完善的道路，这就是事物发展中的否定之否定规律。
    """,
    """
辩证否定观的基本内容是： 第一， 否定是事物的自我否定，是事物内部矛盾运动的结果。
第二，否定是事物发展的环节。第三，否定是新旧事物联系的环节。第四，辩证否定的实质
是“扬弃”，即新事物对旧事物既批判又继承，既克服其消极因素又保留其积极因素。
否定之否定是两次否定、三个阶段的完整过程与统一，事物的辩证发展过程经过肯定—
否定—否定之否定三个阶段。
    """,
    """
考点7 实践与认识
（1）实践的基本形式：物质生产实践、社会政治实践、科学文化实践。
（2）实践的结构：主体——从事实践和认识活动的人；客体——主体活动的对象；中
介——主体用于客体的工具、手段或程序和方法。
（3）辩证唯物主义认识论认为，认识的本质是主体在实践基础上对客体的能动反映。
（4）认识的第一次飞跃是从感性认识到理性认识，第二次飞跃是从理性认识到实践。
（5）感性认识具有直接性和具体性的特点，是认识发展的低级阶段，包括感觉、知觉
和表象三种形式；理性认识具有间接性和抽象性的特点，是认识发展的高级阶段， 包括概念、
判断、推理三种形式。
（6）认识运动的总规律公式是实践—认识—实践。
    """,
    """
考点8 真理与价值
（1）马克思主义认为，真理是标志主观与客观相符合的哲学范畴，是对客观事物及其
规律的正确反映。客观性是真理的本质属性。
（2）任何真理都是绝对性和相对性的统一，二者相互联系、不可分割。
（3）实践是检验真理的唯一标准。
    """,
    """
考点9 社会存在和社会意识
（1）自然地理环境是人类社会生存和发展永恒的、必要的条件，是人们生活和生产的
自然基础。
（2）人口因素也是重要的社会物质生活条件，对社会发展起着制约和影响的作用。
（3）生产方式是社会历史发展的决定力量。
（4）社会存在决定社会意识，社会意识对社会存在具有能动的反作用。
    """,
    """
考点10 生产力与生产关系的矛盾运动及其规律
（1）生产力基本要素包括：劳动资料、劳动对象和劳动者。科学技术是生产力中的重
要因素。
（2）生产关系包括生产资料所有制关系、生产中人与人的关系和产品分配关系。在生
产关系中，生产资料所有制关系是最基本的，它是人们进行物质资料生产的前提，生产、分
配、交换和消费关系在很大程度上是由这种前提决定的，所以生产资料所有制关系是最基本
的、具有决定意义的方面。它是区分不同生产方式、判定社会经济结构性质的客观依据。
（3）生产力决定着生产关系，生产关系对生产力具有反作用，在生产力与生产关系之
间存在着自始至终的矛盾运动。
（4）生产关系一定要适合生产力状况的规律。
    """,
    """
考点11 经济基础与上层建筑的矛盾运动及其规律
（1）上层建筑由意识形态和政治法律制度及设施、政治组织等两部分构成。意识形态
又称观念上层建筑，包括政治法律思想、道德、艺术、宗教、哲学等思想观点；政治法律制
度及设施和政治组织又称为政治上层建筑，包括国家政治制度、立法司法制度和行政制度，
国家政权机构、政党、军队、警察、法庭、监狱等政治组织形态和设施。
（2）经济基础决定上层建筑的产生、性质、变革，上层建筑以服务的形式反作用于经
济基础，上层建筑与经济基础之间始终存在从基本适合到基本不适合再到新的基本适合的矛
盾运动，这些就构成了上层建筑必须适合经济基础状况的规律。
    """,
    """
考点12 社会基本矛盾在历史发展中的作用
（1）生产力和生产关系、经济基础和上层建筑的矛盾是社会基本矛盾。
（2）社会基本矛盾是社会发展的根本动力。
（3）现阶段，我国社会主要矛盾已经转化为人民日益增长的美好生活需要和不平衡不
充分的发展之间的矛盾。
    """,
    """
考点13 阶级斗争、社会革命和改革在社会发展中的作用
（1）阶级斗争是阶级社会发展的直接动力。
（2）社会革命是历史的火车头。
（3）改革是推动社会发展的重要动力。
    """,
    """
第二部分 毛泽东思想和中国特色社会主义理论体系概论
考点1 毛泽东思想的科学内涵、主要内容和活的灵魂
（1）毛泽东思想是马克思主义中国化的第一个重大理论成果。
（2）毛泽东思想的主要内容包括：
①新民主主义革命理论。
②社会主义革命和社会主义建设理论。
③关于革命军队建设、军事战略和国防建设的理论。
④关于政策和策略的理论。
⑤关于思想政治工作和文化工作的理论。
⑥关于党的建设理论。
（3）毛泽东思想的活的灵魂，它们有三个基本方面，即实事求是，群众路线，独立自
主。
    """,
    """
考点2 近代中国国情
（1）近代中国，已经沦为一个半殖民地半封建性质的社会，这是最基本的国情。
（2）近代中国半殖民地半封建的社会性质，决定了社会主要矛盾是帝国主义和中华民
族的矛盾、封建主义和人民大众的矛盾。
（3）近代中国社会的性质和主要矛盾，决定了近代中国革命的根本任务是推翻帝国主
义、封建主义和官僚资本主义的统治，从根本上推翻反动腐朽的政治上层建筑，变革阻碍生
产力发展的生产关系。
    """,
    """
考点3 新民主主义革命的总路线和基本纲领
（1）新民主主义革命总路线是：无产阶级领导的，人民大众的反帝、反封建、反官僚
资本主义的革命。
（2）新民主主义革命的政治纲领是：推翻帝国主义和封建主义的统治，建立一个无产
阶级领导的，以工农联盟为基础的，各革命阶级联合专政的新民主主义的共和国。
（3）新民主主义革命的经济纲领是：没收封建地主阶级的土地归农民所有，没收官僚
资产阶级的垄断资本归新民主主义的国家所有，保护民族工商业。
（4）新民主主义革命的文化纲领是：无产阶级领导的，人民大众的反帝反封建的文化，
即民族的科学的大众的文化。
（5）中国走农村包围城市、武装夺取政权的道路具有现实的可能性。
①最根本原因：中国是一个政治经济发展极不平衡的半殖民地半封建的大国。
②客观原因：国民革命政治影响的存在和全国革命形势继续向前发展。
③主观原因：相当力量的正式红军的存在，共产党组织的有力量和政策的正确。
    """,
    """
考点4 从新民主主义到社会主义的转变
（1）新民主主义社会是一个过渡性的社会。
（2）党在这个过渡时期的总路线和总任务是，要在一个相当长的时期内，逐步实现国
家的社会主义工业化，并逐步实现国家对农业、手工业和资本主义工商业的社会主义改造。
    """,
    """
考点5 社会主义制度在中国的确立
（1）1956年社会主义改造的基本完成，标志着社会主义基本制度在我国初步确立，我
国进入社会主义初级阶段。
（2）社会主义基本制度的确立是中国历史上最深刻最伟大的社会变革，为当代中国一
切发展进步奠定了制度基础，使广大劳动人民真正成为国家的主人和社会生产资料的主人。
    """,
    """
考点6 初步探索的重要理论成果
（1）《论十大关系》标志着党探索中国社会主义建设道路的良好开端。
（2）毛泽东的社会主义矛盾学说分为两大部分：一是关于社会主义社会基本矛盾的理
论，这是社会主义社会矛盾学说的基础；二是关于正确区分和处理两类不同性质矛盾的理论，
这是社会主义矛盾学说的主体。
    """,
    """
考点7 邓小平理论的基本问题和主要内容
（1）邓小平理论就是紧紧围绕“什么是社会主义、怎样建设社会主义”这个问题形成
和发展起来的。
（2）社会主义的本质，是解放生产力，发展生产力，消灭剥削，消除两极分化，最终
达到共同富裕。
（3）邓小平理论的主要内容包括：
①解放思想、实事求是的思想路线。
②社会主义初级阶段理论。
③党的基本路线。
④社会主义根本任务的理论。
⑤“三步走”战略。
⑥改革开放理论。
⑦社会主义市场经济理论。
⑧“两手抓，两手都要硬” 。
⑨“一国两制”。
⑩中国问题的关键在于党。
    """,
    """
考点8 “三个代表”重要思想的核心观点和主要内容
（1）“三个代表”重要思想的核心观点包括：
①中国共产党必须始终代表中国先进生产力的发展要求。
②中国共产党必须始终代表中国先进文化的前进方向。
③中国共产党必须始终代表中国最广大人民的根本利益。
（2）“三个代表”重要思想的主要内容包括：
①发展是党执政兴国的第一要务。
②建立社会主义市场经济体制。
③全面建设小康社会。
④建设社会主义政治文明。
⑤推进党的建设新的伟大工程。
    """,
    """
考点9 科学发展观的科学内涵和主要内容
（1）科学发展观，第一要义是发展，核心立场是以人为本，基本要求是全面协调可持
续，根本方法是统筹兼顾。
（2）科学发展观的主要内容包括：
①加快转变经济发展方式。
②发展社会主义民主政治。
③推进社会主义文化强国建设。
④构建社会主义和谐社会。
⑤推进生态文明建设。
⑥全面提高党的建设科学化水平。
    """,
    """
考点10 习近平新时代中国特色社会主义思想的主要内容
（1） 坚持和发展中国特色社会主义是习近平新时代中国特色社会主义思想的核心要义。
（2）“八个明确”点明了习近平新时代中国特色社会主义思想的主要内容。
①明确坚持和发展中国特色社会主义，总任务是实现社会主义现代化和中华民族伟大
复兴，在全面建成小康社会的基础上，分两步走在21世纪中叶建成富强民主文明和谐美丽
的社会主义现代化强国。
②明确新时代我国社会主要矛盾是人民日益增长的美好生活需要和不平衡不充分的发
展之间的矛盾，必须坚持以人民为中心的发展思想，不断促进人的全面发展、全体人民共同
富裕。
③明确中国特色社会主义事业总体布局是“五位一体”、战略布局是“四个全面”，强调坚
定道路自信、理论自信、制度自信、文化自信。
    """,
    """
④明确全面深化改革总目标是完善和发展中国特色社会主义制度、推进国家治理体系和
治理能力现代化。
⑤明确全面推进依法治国总目标是建设中国特色社会主义法治体系、建设社会主义法治
国家。
⑥明确党在新时代的强军目标是建设一支听党指挥、能打胜仗、作风优良的人民军队，
把人民军队建设成为世界一流军队。
⑦明确中国特色大国外交要推动构建新型国际关系，推动构建人类命运共同体。
⑧明确中国特色社会主义最本质的特征是中国共产党领导，中国特色社会主义制度的最
大优势是中国共产党领导，党是最高政治领导力量，提出新时代党的建设总要求，突出政治
建设在党的建设中的重要地位。
    """,
    """
（3）“十四个坚持”也就是新时代中国特色社会主义基本方略。即坚持党对一切工作
的领导，坚持以人民为中心，坚持全面深化改革，坚持新发展理念，坚持人民当家作主，坚
持全面依法治国，坚持社会主义核心价值体系，坚持在发展中保障和改善民生，坚持人与自
然和谐共生，坚持总体国家安全观，坚持党对人民军队的绝对领导，坚持“一国两制”和推
进祖国统一，坚持推动构建人类命运共同体，坚持全面从严治党。
    """,
    """
考点11 实现中华民族伟大复兴的中国梦
（1）坚持和发展中国特色社会主义的总任务，是实现社会主义现代化和中华民族伟大
复兴，在全面建成小康社会的基础上，分两步走在21世纪中叶建成富强民主文明和谐美丽
的社会主义现代化强国。
（2）中国梦的本质是国家富强、民族振兴、人民幸福。
    """,
    """
考点12 实现社会主义现代化强国“两步走”战略的具体安排
从2020年到21世纪中叶可以分两个阶段来安排：第一个阶段，从 2020 年到 2035年，
在全面建成小康社会的基础上，再奋斗15年，基本实现社会主义现代化。第二个阶段，从
2035年到 21 世纪中叶，在基本实现现代化的基础上，再奋斗 15年，把我国建成富强民主
文明和谐美丽的社会主义现代化强国。
    """,
    """
考点13 发展社会主义民主政治
（1）人民民主专政是我们国家的国体，实质上是无产阶级专政，又是具有中国特色社
会主义的无产阶级专政。
（2）人民代表大会制度是我国的政体。
（3）“一国两制”即“一个国家，两种制度”的简称。“一国两制”指的是在中华人
民共和国内，国家的主体坚持社会主义制度，同时在香港、澳门、台湾保持原有的资本主义
制度长期不变。
    """,
    """
考点14 推动社会主义文化繁荣兴盛
（1）社会主义核心价值体系包括四个方面的基本内容：坚持马克思主义指导思想，是
社会主义核心价值体系的灵魂；树立中国特色社会主义的共同理想，是社会主义核心价值体
系的主题；弘扬民族精神和时代精神，是社会主义核心价值体系的精髓；树立社会主义荣辱
观，是社会主义核心价值体系的基础。
（2）培育和践行社会主义核心价值观的主要内容包括：富强、民主、文明、和谐，自
由、平等、公正、法治，爱国、敬业、诚信、友善。
    """,
    """
考点15 全面深化改革
（1）全面深化改革总目标是完善和发展中国特色社会主义制度、推进国家治理体系和
治理能力现代化。
（2）正确处理全面深化改革中的重大关系：处理好解放思想和实事求是的关系；处理
好整体推进和重点突破的关系；处理好顶层设计和摸着石头过河的关系；处理好胆子要大和
步子要稳的关系；处理好改革、发展、稳定的关系。
    """,
    """
考点16 坚持和平发展道路
（1）和平与发展是当今时代的主题。
（2）霸权主义、强权政治是当代战争的主要根源和世界发展的主要障碍。
（3）中国独立自主的和平外交政策，就是把国家主权和安全放在第一位。
（4）中国坚定不移地奉行独立自主的和平外交政策，走和平发展道路，这是由我国的
社会主义性质和在国际上的地位所决定的。
（5）推动建设相互尊重、公平正义、合作共赢的新型国际关系。
    """,
    """
考点17 推动构建人类命运共同体
构建人类命运共同体思想的核心就是“建设持久和平、普遍安全、共同繁荣、开放包
容、清洁美丽的世界”。
（1）政治上，要相互尊重、平等协商，坚决摒弃冷战思维和强权政治，走对话而不对
抗、结伴而不结盟的国与国交往新路。建设一个持久和平的世界，根本要义在于国家之间要
构建平等相待、互商互谅、互学互鉴的伙伴关系。
（2）安全上，要坚持以对话解决争端、以协商化解分歧，统筹应对传统和非传统安全
威胁，反对一切形式的恐怖主义。
（3）经济上，要同舟共济，促进贸易和投资自由化便利化，推动经济全球化朝着更加
开放、包容、普惠、平衡、共赢的方向发展。
（4）文化上，要尊重世界文明多样性，以文明交流超越文明隔阂、文明互鉴超越文明
冲突、文明共存超越文明优越。
（5）生态上，要坚持环境友好，合作应对气候变化，保护好人类赖以生存的地球家园。
    """,
    """
考点18 坚持党对一切工作的领导
（1）党是最高政治领导力量：这是由中国共产党的先进性决定的；这是由中国共产党
的历史作用赢得的；这是由中国共产党勇于自我革命的品格铸就的。
（2）全面增强党的学习、政治领导、改革创新、科学发展、依法执政、群众工作、狠
抓落实、驾驭风险本领。
    """,
]

for i_txt, file_text in enumerate(file_texts):
    new_file_path = f'audio_files/tmp/aji_{i_txt}.mp3'
    if not os.path.exists(new_file_path):
        gTTS(file_text, lang='zh').save(new_file_path)
        print(f"{new_file_path} saved")
    else:
        print(f"{new_file_path} ALREADY EXISTS")

In [235]:
all_audio_files = []
for i_txt, file_text in enumerate(file_texts):
    new_file_path = f'audio_files/tmp/aji_{i_txt}.mp3'
    all_audio_files.append(AudioSegment.from_mp3(new_file_path))
print(len(all_audio_files))
combined = all_audio_files[0]
for audio in all_audio_files[1:]:
    combined += audio
combined.export(f"audio_files/tmp/aji_concat.mp3", format="mp3")

38


<_io.BufferedRandom name='audio_files/tmp/aji_concat.mp3'>

# 7. english pdf

In [238]:
file_texts = [
    """
考点1 元音字母在单词中的读音规则
英语中共有26 个字母，其中元音字母有 5 个，分别是 a, e, i, o, u。
    """,
    """
字母 音节种类 读音 例词
    """,
    """
a
在重读开音节中 [eɪ] blame, cake, state, take
在重读闭音节中 [æ] sad, cat, cap, map, stand
在 r 音节中 [ɑː] cart, large, star, harm
在非重读音节中 [ə][ɪ] pacific, cabbage
    """,
    """
e
在重读开音节中 [iː] be, meter, these, we
在重读闭音节中 [e] bed, desk, kept, let, melt
在 r 音节中 [ɜː] her, nerve, serve, term
在非重读音节中 [ə][ɪ] golden, open, wanted
    """,
    """
i
在重读开音节中 [aɪ] five, life, pipe, white, by
在重读闭音节中 [ɪ] bring, gift, sit, stick, win
在 r 音节中 [ɜː] first, girl, shirt, third
在非重读音节中 [ɪ][aɪ] office, morning, satisfy
ɪ ]
    """,
    """
o
在重读开音节中 [əʊ] close, hope, smoke, zone
在重读闭音节中 [ɒ] box, fog, long, pot, shop
在 r 音节中 [ɔː] formal, morning, north, perform
在非重读音节中 [ə][əʊ] collective, photo
    """,
    """
u
在重读开音节中 [juː] huge, music, student, use
在重读闭音节中 [ʌ] cup, cut, dust, nut, sun
在 r 音节中 [ɜː] burn, nurse, purpose, turn
在非重读音节中 [ə][ju][juː] difficult, occupy, attitude
    """,
    """
考点2 辅音字母在单词中的读音规则
26 个英文字母中，除了 a, e, i, o, u 这5 个元音字母，其余的字母都是辅音字母。
字母 在音节中的位置 读音 例词
    """,
    """
b
在词首或词尾 [b] book, beef, club
在词尾，前有 m 不发音 climb, comb, tomb
    """,
    """
c
在元音字母 a, o, u 前 [k] copy, cut, cat, come
在 e, i(y)前 [s] device, decide, bicycle
在 ia, ie, io 前 [ʃ] social, conscious
    """,
    """
g
常见读音 [ɡ] egg, game, globe
在 e, i(y)前 [dʒ ] huge, college, technology
    """,
    """
h 常见读音 [h] hair, high, hand
在部分单词词首或中间 不发音 hour, honour, exhibition
    """,
    """
s
在词首或在清辅音前后 [s] six, sit, system
在两个元音字母之间 [z] design, reason, music
在元音字母和不发音字母 e 之间 名词中读[s]，动词中读[z]
close
    """,
    """
t
常见读音 [t] hot, ten
在 ure 前及个别词中 [tʃ] picture, question
在 ia, ie, io 前 [ʃ] patient, station
在个别词中 不发音 listen, whistle
    """,
    """
n
常见读音 [n] neck, nod
在 k, g 前 [ŋ ] tank, anger
在 mn 字母组合中 不发音 autumn
    """,
    """
x
常见读音 [ks] next, fix, fox
在重读元音前 [ɡz] exact, exam, exhaust
    """,
    """
k
常见读音 [k] kind, kick, kill
在 kn 字母组合中 不发音 knife, know
    """,
    """
w
常见读音 [w] watch, woman, want
在 wh, wr 字母组合中及在个别词中 不发音 whole, who, write, answer
    """,
    """
y 在元音字母前 [j] year, young, yield
    """,
#     """
# 第二部分 词法知识
# 考点1 名词
# 1.名词的复数形式
# 可数名词复数的规则变化
#     """,
]

"""
规则 例词
一般情况下直接在词尾加-s room—rooms, apple—apples
以-s, -x, -ch, -sh 结尾的在词尾加-es glass—glasses, fox—foxes
以“辅音字母+y”结尾的变 y 为 i，再加-es baby—babies, library—libraries
以-o 结尾 以“辅音字母+o”结尾的在词尾加-es potato—potatoes
以“元音字母+o”结尾的在词尾加-s zoo—zoos, radio—radios
以-o 结尾的部分外来词和缩略词词尾加-s piano—pianos, photo—photos
以-f 或-fe 结尾 以-f 或-fe 结尾的部分可数名词变 f 或 fe 为 v，再加-es leaf—leaves, wife—wives
以-f 或-fe 结尾的部分可数名词词尾直接加-s proof—proofs, belief—beliefs
可数名词复数的不规则变化
规则 例词
改变名词中的元音字母 foot—feet, man—men
单复数同形 sheep, deer, fish, means
特殊形式 child—children, ox—oxen
复
合
名
词
变
复
数
一些以 man, woman 结尾的合成词变复数时，与 man, woman 的变化
形式相同
policeman—policemen
man, woman 和其他名词构成的复合名词变复数时，两个名词全都
要变成复数
man teacher—men teachers
“可数名词+介词（短语）”构成的复合名词变复数时，前面名词变
复数
passer-by—passers-by
“动词/过去分词+副词”构成的复合名词变复数时，词尾加-s grown-up—grown-ups
2.名词所有格
名词所有格主要有-
’s 属格、of 属格和双重所有格三种形式。-
’s 属格多用于有生命的东
西，of 属格多用于无生命的东西。
（1）-
’s 属格
表示有生命的名词的所有格一般在词尾加-’s，以-s 或-es 结尾的名词只加-
’，如 my
brother’s book, the heroes’ names。 两人或多人共有某人或某物时，只在最后一个人名后加-’s，
版权所有∙翻版必究
4
如果该人或物为各自所有，则每个人名后都要加-’s，如 Lucy and Lily’s room （共有的） , Lucy’s
and Lily’s room（不共有） 。表示时间、距离、国家、组织机构等无生命事物的名词，也可以
在词尾加-’s 或者-
’构成所有格，如today’s newspaper, ten minutes’ drive。表示诊所、店铺、
医院等场所时，-’s 所有格后常常不出现其所修饰的名词，如 at the doctor’s(在诊所), at my
sister’s（在我姐姐家）。
（2）of 属格
of 属格的构成通常为“名词 A+of+名词 B”，表示“B 的A”。of 属格多表示无生命物体
的所属关系，有时也可表示有生命物体的所属关系。
（3）双重所有格
上面所讲的两种所有格的形式可以结合起来，构成双重所有格，即“of+名词所有格”，
表示部分观念或感情色彩。
考点2 动词
1.连系动词
连系动词不能单独作谓语，必须和表语一起构成系表结构来说明主语的状态、性质、特
征等情况。最常用的连系动词是 be(是)，其他常见的连系动词还有 seem, appear, feel, look,
sound, taste, smell, become, turn, grow, get, go, fall, remain, prove。
2.助动词
助动词本身没有独立的意义，不能单独作谓语，必须和实义动词或连系动词的某种形式
一起构成谓语。常用的助动词有 do, be, have, shall(should), will(would)等。
3.情态动词
常见的情态动词有 can, could, may, might, must, need, shall, should, will, would, ought to,
dare 等。must 表示“必须，必要”，用于一般疑问句中时，肯定回答用 must，否定回答用
needn’t 或don’t have to。mustn’t 表示“不允许，禁止”。“could+have done”表示过去本能
够做而实际上未做某事。“should+have done”表示过去本应该做而实际上未做某事。
“must+have done”表示对过去发生情况的肯定推测。
4.动词的时态
英语中共有16 种时态，其中常用的有以下 9 种。
时态 结构
一般过去时 主语+动词过去式
一般现在时 主语+动词原形（若主语为第三人称单数，动词用第三人称单数形式）
一般将来时 主语+will/shall+动词原形
过去进行时 主语+was/were+现在分词
现在进行时 主语+am/is/are+现在分词
将来进行时 主语+will/shall+be+现在分词
过去完成时 主语+had+过去分词
现在完成时 主语+have/has+过去分词
将来完成时 主语+will/shall+have+过去分词
考点3 代词
1.指示代词
表示“这个”“那个”“这些”“那些”等概念的词叫作指示代词。指示代词有this, that, these,
those, such, same, it。 英语中有时为了避免重复，可用that 或those 代替前面提过的人或事物。
same 之前一般须用定冠词the。
2.关系代词
关系代词有who, whose, whom, that, which 等，都用来引导定语从句。它们在定语从句
版权所有∙翻版必究
5
中可作主语、宾语、定语或表语，代表主句中为定语从句所修饰的先行词。
3.不定代词
不定代词大体可分为两类。一类是兼作代词和限定词的不定代词，如 all, another, both,
each, either, many, much, neither, few, little 等；另一类是由any, every, no, some 后加-body, -one,
-thing 构成的合成不定代词，如 everything, someone, anybody。both 意为“两者都”，either
意为“两个中的任何一个”，neither 意为“两者中没有一个”。either 和 neither 作定语，后面
名词用单数；either 和 neither 作主语，谓语动词用单数形式。
4.it 的用法
it 用以代替上文提到过的人或事物， 用以代替上文的 this 和that， 用以表示时间、日期、
地点、温度、距离、环境等，用以作形式主语、形式宾语，还可以用于强调结构中。
考点4 形容词与副词
1.形容词与副词的比较级和最高级
英语中大多数形容词与副词都有原级、比较级和最高级。比较级形式是“原级+-er”或
“more+原级” ；最高级形式是“原级+-est”或“most+原级”
。
2.常考的形容词比较级句型结构
（1）表示A 超过B：A+be+形容词比较级+than+B
（2）表示A 和B 是同等程度：A+be+as+形容词原级+as+B
（3）越„„，越„„：the+比较级…，the+比较级…
考点5 固定搭配
attribute...to....把某事归因于„„
be good at 在„„方面做得好
be interested in 对„„感兴趣
be responsible for 对„„负责
be satisfied with 对„„满意
can (could) not help 禁不住，忍不住
come up with 提出，想出
deal with 对付，处理；涉及，论述
depend on/upon 依靠
fall behind 落后，落在„„的后面
feel like (doing) sth.想要某物，想做某事
find out 找出，查明，发现
get rid of 摆脱；除去；处理掉
had better 最好，还是„„为好
in case 假使；以防万一
in danger 处于危险中
in order to 为了；以便
in spite of 尽管；不顾；虽然
in turn 依次，轮流
instead of 代替，而不是
keep in mind 记住，不忘记
keep up with 与„„同步，跟上
lead to 通向；导致，引起
leave behind 留下，忘记带走
make sense 有意义；合理，讲得通
make up one’s mind 下决心，决定
make use of 利用，使用
no more than 不过，仅仅
on behalf of 代表，为了
on purpose 故意地，有意地
on the contrary 正相反，反之
pay attention to 重视，注意
play a part in 在„„中起作用，参与
put off 推迟，延期；拖延
rather than 而不是
regard as 把„„看作，把„„认为是
regardless of 不管，不顾
result in 导致，引起，结果为
so as to 以便；以致
stand for 代表，象征，意味着
take advantage of 利用
take for granted 想当然；认为理当如此
take into account 考虑，斟酌
take part in 参加，参与
take place 产生，发生；进行，举行
the more…the more…愈„„愈„„
try one’s best 竭尽全力，尽力
turn out 结果是，证明是
what if 如果„„将会怎样
work out 算出；解决
版权所有∙翻版必究
6
第三部分 句法知识
考点1 主谓一致
1.名词作主语时的主谓一致
集体名词作主语若强调整体，谓语动词用单数；若突出个体，谓语动词用复数。表示时
间、金钱等的词语作主语，通常被看作一个整体，谓语动词用单数。
2.并列主语的主谓一致
（1）and 连接并列主语，谓语动词通常用复数，但当并列主语指的是同一个人、事物
或概念，谓语动词用单数。
（2）or, either…or…, neither…nor…, not only…but also…等连接并列主语时遵循就近原
则。
（3） as well as, rather than, but, together with, along with, with 等连接并列主语时，谓语动
词的数与最前面的主语保持一致。
考点2 非谓语动词
1.动词不定式
时态 主动形式 被动形式
一般式 to do to be done
完成式 to have done to have been done
进行式 to be doing —
完成进行式 to have been doing —
2.现在分词
时态 主动形式 被动形式
一般式 doing being done
完成式 having done having been done
3.过去分词
规则动词的过去分词一般是由动词加-ed 构成。过去分词在句子中可以作定语、表语、
状语、宾补。
4.动名词
动名词在句子中可以作主语、宾语、表语、定语等。动名词的基本形式如下：
考点3 虚拟语气
虚拟语气在条件从句中的构成
虚拟条件句 主句
与现在事实相反 If+主语+动词过去式（be 动词用were） 主语+would/should/could/might＋动词原形
与过去事实相反 If+主语+had＋过去分词 主语+would/should/could/might＋have＋过去分
词
与将来事实相反 If+主语+动词过去式（be 动词用were）
If+主语+should/were to+动词原形
主语+would/should/could/might＋动词原形
考点4 名词性从句
1.主语从句
时态 主动形式 被动形式
一般式 doing being done
完成式 having done having been done
版权所有∙翻版必究
7
主语从句在复合句中充当主语，通常放在主句谓语动词之前或由形式主语 it 代替，而
主语从句本身放在句子末尾。主语从句的引导词有 that, whether, what, who, which, how, why,
when, where 等。当主语从句作主语时，谓语动词一般用第三人称单数形式。
2.宾语从句
宾语从句在复合句中作宾语，通常放在主句谓语动词（及物动词）、形容词或介词之后。
若主句谓语动词为think, consider, suppose, believe 等，其后的宾语从句若含有否定意义，一
般要把否定词转移到主句谓语上，从句谓语用肯定式。
3.表语从句
表语从句在复合句中作表语，放在系动词之后，一般结构是“主语+系动词+表语从句”。
可以接表语从句的系动词有be, look, remain, seem 等。 引导表语从句的that一般不可省略。
4.同位语从句
同位语从句在复合句中作名词的同位语，对前面的名词起进一步说明的作用，通常用于
fact, news, idea, hope, belief, thought, doubt, truth 等名词之后。
考点5 定语从句
引导定语从句的关系代词有 that, which, who（宾格whom，所有格 whose）和关系副词
when, why, where。其中，who, whom 指人；which 指物；that 既可以指人也可以指物；when
指时间；where 指地点；why 指原因。
关系代词只能用that 的情况：
（1）先行词为all, everything, anything, nothing 等不定代词或由every, all, any, little,
few, much 等修饰时，关系代词用 that。
（2）先行词前有序数词、形容词最高级或 last, next, the only 等修饰时，关系代词用 that。
（3）先行词既有人又有物时，关系代词用that。
考点6 状语从句
（1）as 引导的让步状语从句如果位于主句之前，须用倒装语序。从句中的表语、状语
或动词原形置于句首。若表语是名词，前置时要省略冠词。
（2）no sooner...than...与 hardly/scarcely...when...连接句子时，主句谓语动词应用过去
完成时，从句中谓语动词应用一般过去时。当 no sooner 和 hardly/scarcely 位于句首时，应
用倒装语序。
考点7 强调句
it 引导的强调句的基本形式是“It is/was+被强调部分+that/who…”， 可以用来强调主语、
宾语、状语等，不能强调谓语动词； 一般疑问句句型是“Is/Was it+被强调部分+that/who…?”；
特殊疑问句句型是“特殊疑问词+is/was it that…?”。
考点8 倒装句
1.全部倒装
（1）用于there be 句型。
（2）用于“here/there/now/then+不及物动词+主语”的句型中，或以out, in, up, down,
away 等副词开头的句子，以表示强调。
（3）用于以 so/nor/neither 开头的句子，表示重复前句部分内容。
2.部分倒装
（1）在so…that 句型中，若 so…提至句首，则构成部分倒装。
（2）用于“形容词（名词、动词）+as/though”引导的让步状语从句中。
（3） 用于以 never, hardly, rarely, little, by no means 等含有否定意义的词或短语开头的
句子。
（4）用于only 开头的句子（only 后面为副词、介词短语或句子）。
"""

for i_txt, file_text in enumerate(file_texts):
    new_file_path = f'audio_files/tmp/aji_english_{i_txt}.mp3'
    if not os.path.exists(new_file_path):
        gTTS(file_text, lang='zh').save(new_file_path)
        print(f"{new_file_path} saved")
    else:
        print(f"{new_file_path} ALREADY EXISTS")

audio_files/tmp/aji_english_0.mp3 saved
audio_files/tmp/aji_english_1.mp3 saved
audio_files/tmp/aji_english_2.mp3 saved
audio_files/tmp/aji_english_3.mp3 saved
audio_files/tmp/aji_english_4.mp3 saved
audio_files/tmp/aji_english_5.mp3 saved
audio_files/tmp/aji_english_6.mp3 saved
audio_files/tmp/aji_english_7.mp3 saved
audio_files/tmp/aji_english_8.mp3 saved
audio_files/tmp/aji_english_9.mp3 saved
audio_files/tmp/aji_english_10.mp3 saved
audio_files/tmp/aji_english_11.mp3 saved
audio_files/tmp/aji_english_12.mp3 saved
audio_files/tmp/aji_english_13.mp3 saved
audio_files/tmp/aji_english_14.mp3 saved
audio_files/tmp/aji_english_15.mp3 saved
audio_files/tmp/aji_english_16.mp3 saved
audio_files/tmp/aji_english_17.mp3 saved
audio_files/tmp/aji_english_18.mp3 saved


In [239]:
print(len(all_audio_files))
all_audio_files = []
for i_txt, file_text in enumerate(file_texts):
    new_file_path = f'audio_files/tmp/aji_english_{i_txt}.mp3'
    all_audio_files.append(AudioSegment.from_mp3(new_file_path))

combined = all_audio_files[0]
for audio in all_audio_files[1:]:
    combined += audio
combined.export(f"audio_files/tmp/aji_english_concat18.mp3", format="mp3")

19


<_io.BufferedRandom name='audio_files/tmp/aji_english_concat18.mp3'>